# 01. Setup and Load Data

### 1.1 Dependencies Install and Setup

In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
from mtcnn.mtcnn import MTCNN

2024-02-03 13:59:05.419966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/n7/qn8s75510p5chs7mw2tvcgy80000gp/T/ipykernel_4996/259250003.py:4: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


### 1.2 Target directories and data load

##### Call the saved model 1.

In [2]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
model_folder = os.path.join(parent_directory, "model")
model_path = os.path.join(model_folder, "parfois_product_feature_classification.h5")

parfois_model_1 = load_model(model_path)

##### Check for a xlsx file.

In [3]:
product_ref_path = os.path.join("product_ref_call", "product_ref.xlsx")

In [4]:
if not os.path.exists(product_ref_path):
    print("File Error: The file does not exist.", product_ref_path)
    exit()
try:
    product_ref_df = pd.read_excel(product_ref_path)
except Exception as e:
    print("Error reading the Excel file:", e)
    exit()

##### DF creation to store the prediction of the first model.

In [7]:
downloaded_images_path = "studio_folder"  
images_to_check = set(os.listdir(downloaded_images_path))
images_to_check = pd.DataFrame(images_to_check)
images_to_check.rename(columns={0: "image_file_name"}, inplace=True)
images_to_check["model_1_images_predictions"] = np.nan

images_to_check

,image_file_name,model_1_images_predictions
0,217013_BK_3y.jpg,NaN
1,218093_BK_4y.jpg,NaN
2,217013_FU_5y.jpg,NaN
3,216943_BM_5y.jpg,NaN
4,216943_BM_1y.jpg,NaN
...,...,...
70,216875_FU_2y.jpg,NaN
71,216786_FU_5y.jpg,NaN
72,217044_PK_3y.jpg,NaN
73,216780_FU_5y.jpg,NaN


# 02. Model 1 Call and Run

In [8]:
parent_directory = os.getcwd()

def update_predictions(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(parent_directory, "studio_folder", img_file_name)
    img_to_check = cv2.imread(img_path)

    if img_to_check is not None:
        resize = tf.image.resize(np.expand_dims(img_to_check, 0), (256, 256))[0]
        prediction = parfois_model_1.predict(np.expand_dims(resize / 255, 0))

        if prediction > 0.5:
            return 1
        else:
            return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None 

images_to_check["model_1_images_predictions"] = images_to_check.apply(update_predictions, axis=1)

1/1 [==============================] - 0s 25ms/step


In [9]:
images_to_check

,image_file_name,model_1_images_predictions
0,217013_BK_3y.jpg,0
1,218093_BK_4y.jpg,0
2,217013_FU_5y.jpg,0
3,216943_BM_5y.jpg,0
4,216943_BM_1y.jpg,1
...,...,...
70,216875_FU_2y.jpg,1
71,216786_FU_5y.jpg,1
72,217044_PK_3y.jpg,1
73,216780_FU_5y.jpg,1


# 03. Model 2 Call and Run

In [10]:
images_to_check["model_2_images_predictions"] = np.nan

In [11]:
parfois_model_2 = MTCNN(min_face_size=30, 
                               steps_threshold=[0.7, 0.8, 0.8])

In [12]:
def model_predictions_2(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        detected_faces = parfois_model_2.detect_faces(img_to_check)
        
        for face in detected_faces:
            if "keypoints" in face and "left_eye" in face["keypoints"] and "right_eye" in face["keypoints"] and "mouth_left" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1 
            if "keypoints" in face and "left_eye" in face["keypoints"] and "mouth_left" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "left_eye" in face["keypoints"]:
                return 1  
        return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return -1

images_to_check["model_2_images_predictions"] = images_to_check.apply(model_predictions_2, axis=1)

1/1 [==============================] - 0s 19ms/step


In [14]:
images_to_check

,image_file_name,model_1_images_predictions,model_2_images_predictions
0,217013_BK_3y.jpg,0,0
1,218093_BK_4y.jpg,0,0
2,217013_FU_5y.jpg,0,0
3,216943_BM_5y.jpg,0,0
4,216943_BM_1y.jpg,1,0
...,...,...,...
70,216875_FU_2y.jpg,1,0
71,216786_FU_5y.jpg,1,0
72,217044_PK_3y.jpg,1,0
73,216780_FU_5y.jpg,1,0


# 03. Update the product ref. xlsx file on the product_ref_call

### 3.1 Data cleaning for the output data frame 

In [16]:
images_to_check.loc[(images_to_check["model_1_images_predictions"] == 0) & (images_to_check["model_2_images_predictions"] == 0), "parfois_app_output"] = "Product"
images_to_check.loc[(images_to_check["model_1_images_predictions"] == 1) & (images_to_check["model_2_images_predictions"] == 0), "parfois_app_output"] = "No_Recognition"
images_to_check.loc[(images_to_check["model_1_images_predictions"] == 1) & (images_to_check["model_2_images_predictions"] == 1), "parfois_app_output"] = "Recognition"
images_to_check.loc[(images_to_check["model_1_images_predictions"] == 0) & (images_to_check["model_2_images_predictions"] == 1), "parfois_app_output"] = "Recognition"

In [17]:
images_to_check["image_file_name"] = images_to_check["image_file_name"].apply(lambda x: "_".join(x.split("_")[:2]))

##### Function run to keep only the final answer for every ref., meaning that if there is any picture ref. with human model or partial human model, there was a human model involved.

In [18]:
def process_output(group):
    recognition_row = group[group["parfois_app_output"] == "Recognition"]
    no_recognition_row = group[group["parfois_app_output"] == "No_Recognition"]
    product_row = group[group["parfois_app_output"] == "Product"]

    if not recognition_row.empty:
        return group.loc[[recognition_row.index[0]]]

    elif not no_recognition_row.empty:
        return group.loc[[no_recognition_row.index[0]]]

    elif not product_row.empty:
        return group.loc[[product_row.index[0]]]

In [19]:
results_df = images_to_check.groupby("image_file_name").apply(process_output).reset_index(drop=True)

In [20]:
results_df

,image_file_name,model_1_images_predictions,model_2_images_predictions,parfois_app_output
0,216394_BM,1,1,Recognition
1,216780_FU,1,1,Recognition
2,216786_FU,1,1,Recognition
3,216855_KK,1,1,Recognition
4,216875_FU,1,0,No_Recognition
5,216875_LG,0,0,Product
6,216943_BM,1,1,Recognition
7,217013_BK,1,0,No_Recognition
8,217013_FU,1,0,No_Recognition
9,217022_GY,1,1,Recognition


##### We get 2 errors for the final predictions in parfois_app_output. It's for the ref. 216875_FU and 217022_GY. If we check the images, they have the same model. We can conclude that the model was not trained in a sufficient way for this kind of facial features.

In [21]:
results_df = images_to_check.sort_values(by="image_file_name").reset_index(drop=True)

In [22]:
results_df.to_excel("studio_folder_check_updated.xlsx", index=False)